# A Guided Tour of Ray Core: Remote Functions

[*Remote Functions*](https://docs.ray.io/en/latest/walkthrough.html#remote-functions-tasks)
involve using a `@ray.remote` decorator on a function. 

This implements a [*task parallelism*](https://patterns.eecs.berkeley.edu/?page_id=208) pattern, with properties: *data independence*, *stateless*

---

First, let's start Ray…

In [1]:
from icecream import ic
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

File descriptor limit 256 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'


{'node_ip_address': '192.168.1.65',
 'raylet_ip_address': '192.168.1.65',
 'redis_address': '192.168.1.65:6379',
 'object_store_address': '/tmp/ray/session_2021-02-23_15-22-13_327603_93131/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-23_15-22-13_327603_93131/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-02-23_15-22-13_327603_93131',
 'metrics_export_port': 63361,
 'node_id': '719947d337b6491f89fbaa0ff51e118a2d77f456'}

## Remote Functions example

Set up for this example...

In [2]:
import time
import random

The following is just a regular Python function...

In [3]:
def my_function ():
    return 1

When called, it simply returns an integer:

In [4]:
my_function()

1

However, by adding the `@ray.remote` decorator, a regular Python function becomes a Ray remote function:

In [5]:
@ray.remote
def my_function():
    return 1

To invoke this remote function, use the `remote` method. This will immediately return an object ref (a *future* in Python) and then create a task that will be executed on a worker process.

In [6]:
obj_ref = my_function.remote()

The result can be retrieved with `ray.get`

In [7]:
ray.get(obj_ref)

1

Invocations of Ray *remote functions* happen in parallel, and all computation gets performed in the background, driven by Ray's internal event loop.

To illustrate this, first let's define a relatively "slow" function...

In [8]:
@ray.remote
def slow_function():
  time.sleep(2)
  return random.randint(0, 9)

Now we'll iterate through multiple calls, showing that this does not block:

In [9]:
futures_list = []

for i in range(4):
    future = slow_function.remote()
    futures_list.append(future)
    ic(i)

ic| i: 0
ic| i: 1
ic| i: 2
ic| i: 3


In [10]:
for future in futures_list:
    ic(ray.get(future))

ic| ray.get(future): 5
ic| ray.get(future): 2
ic| ray.get(future): 5
ic| ray.get(future): 2


---
## References

[*Patterns for Parallel Programming*](https://www.goodreads.com/book/show/85053.Patterns_for_Parallel_Programming)  
Timothy G. Mattson, Beverly A. Sanders, Berna L. Massingill  
Addison-Wesley (2004)